## Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [2]:
import pandas as pd
import random
import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, DepthwiseConv1D, BatchNormalization, Activation, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D, Add, ReLU, Dropout, Multiply, Concatenate
from sklearn.metrics import accuracy_score, f1_score
from scipy.fft import fft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from keras.optimizers import Adam

## Data Loading

In [3]:
project_dataset_dir = '/content/drive/MyDrive/EMG/EMG-data.csv'
df = pd.read_csv(project_dataset_dir)

## Feature extraction and engineering

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4237907 entries, 0 to 4237906
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   time      int64  
 1   channel1  float64
 2   channel2  float64
 3   channel3  float64
 4   channel4  float64
 5   channel5  float64
 6   channel6  float64
 7   channel7  float64
 8   channel8  float64
 9   class     int64  
 10  label     int64  
dtypes: float64(8), int64(3)
memory usage: 355.7 MB


In [5]:
# Drop the time column, as it is not necessary
df.drop('time', inplace=True, axis=1)
df.head()

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class,label
0,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001,0,1
1,0.00001,-0.00002,-0.00001,-0.00003,0.00000,-0.00001,0.00000,-0.00001,0,1
2,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1
3,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1
4,-0.00001,0.00001,0.00002,0.00000,0.00001,-0.00002,-0.00001,0.00001,0,1


In [6]:
df.info()
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4237907 entries, 0 to 4237906
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   channel1  float64
 1   channel2  float64
 2   channel3  float64
 3   channel4  float64
 4   channel5  float64
 5   channel6  float64
 6   channel7  float64
 7   channel8  float64
 8   class     int64  
 9   label     int64  
dtypes: float64(8), int64(2)
memory usage: 323.3 MB


4237907

In [7]:
# Check for null dataset
df.isna().sum()

,0
channel1,0
channel2,0
channel3,0
channel4,0
channel5,0
channel6,0
channel7,0
channel8,0
class,0
label,0


In [8]:
# Count the size of each class
print(df['class'].value_counts())

class
0    2725157
6     253009
5     251733
4     251570
1     250055
3     249494
2     243193
7      13696
Name: count, dtype: int64


In [9]:
# drop the 0th class, as it is unmarked data
df.drop(df[df['class'] == 0].index, inplace=True)

In [10]:
df.head()

,channel1,channel2,channel3,channel4,channel5,channel6,channel7,channel8,class,label
2287,-0.00001,0.00000,-0.00001,0.00000,0.00000,-0.00001,-0.00001,0.00001,1,1
2288,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1,1
2289,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1,1
2290,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1,1
2291,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1,1


In [11]:
# Check if all the classes have 36 unique labels
all_labels_present = df.groupby('class')['label'].nunique() == 36
all_labels_present

,label
class,
1,True
2,True
3,True
4,True
5,True
6,True
7,False


In [12]:
# Check how many label in class 7
class_7_data = df[df['class'] == 7]
class_7_label_counts = class_7_data.groupby('label').size().reset_index(name='count')
class_7_label_counts

,label,count
0,11,7105
1,30,6591


In [13]:
# Drop the 7th class
data = df[df['class'] != 7]

In [14]:
# data['class'].value_counts()

In [15]:
# subjects_data = data.groupby(['label','class'])

In [16]:
# Check if the classes are
class_counts = data.groupby('class').size().reset_index(name='count')

print('Class distribution:')
print(class_counts)

Class distribution:
   class   count
0      1  250055
1      2  243193
2      3  249494
3      4  251570
4      5  251733
5      6  253009


In [17]:
# Drop the label column
data.drop('label', inplace=True, axis=1)

<ipython-input-17-2c4f8e338965>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('label', inplace=True, axis=1)


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1499054 entries, 2287 to 4237211
Data columns (total 9 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   channel1  1499054 non-null  float64
 1   channel2  1499054 non-null  float64
 2   channel3  1499054 non-null  float64
 3   channel4  1499054 non-null  float64
 4   channel5  1499054 non-null  float64
 5   channel6  1499054 non-null  float64
 6   channel7  1499054 non-null  float64
 7   channel8  1499054 non-null  float64
 8   class     1499054 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 114.4 MB


In [19]:
# Normalize the data
def normalized(data):
    min_value = np.min(data)
    max_value = np.max(data)
    normalized_data = (data - min_value) / (max_value - min_value)
    return normalized_data

In [20]:
# Sliding window approach to reduce and extract data features
def sliding_window(data, window_size, stride):
    windowed_data = []
    labels = []

    for i in range(0, len(data) - window_size + 1, stride):
        windowed = data.iloc[i:i+window_size, :-1].values
        label = data.iloc[i+window_size-1, -1]
        windowed_data.append(windowed)
        labels.append(label)

    return np.array(windowed_data), np.array(labels)

In [ ]:
# Use the sliding window on the data
window_size = 150 # hyperparameter
stride = 30 # hyperparameter
X, Y = sliding_window(data, window_size, stride)

In [ ]:
# Normalize the X data
X = normalized(X)

In [ ]:
# Make the class start from 0, instead of 1, as Keras accepts [1,6)
Y = Y - 1

## Apply Fourier Transform to the data

In [ ]:
def fourier_transform(X):
    transformed_data = []

    for item in X:
        transformed_item = []
        for channel in item.T:  # Transpose the data and apply Fourier transform to each channel separately
            fft_values = fft(channel) # This gives amplitude
            transformed_item.append(np.abs(fft_values))  # Only use amplitude information

        transformed_data.append(np.array(transformed_item))

    return np.array(transformed_data)

In [ ]:
# Apply fourier transform to the data
transformed_X = fourier_transform(X)

In [ ]:
transposed_X = np.transpose(transformed_X, (0, 2, 1))

## Do not apply Fourier Transform

In [ ]:
transposed_X = X

# RUN THIS

## Dataset loading and preprocessing

In [ ]:
num_classes = len(np.unique(Y))   # set class size

## Model Definitions

In [ ]:
# CNN
def CNN_model0(input_shape, num_classes):
    model = Sequential()
    # Convolutional layers 1
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPool1D(pool_size=2))
    # Convolutional layers 2
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    # Convolutional layers 3
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    # Flattening layers
    model.add(Flatten())
    # Full connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------3-LAYER-CNN---------------------------------')
    model.summary()
    return model

# VGG
def VGG_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu", input_shape=input_shape))
    model.add(Conv1D(filters=64, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=128, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(Conv1D(filters=512, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(num_classes, activation="softmax"))
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------VGGNET---------------------------------')
    model.summary()
    return model

# ZF
def ZF_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=96, kernel_size=7, strides=2, padding="valid", activation="relu", input_shape=input_shape))
    model.add(MaxPool1D(pool_size=3, strides=2))
    model.add(Conv1D(filters=256, kernel_size=3, strides=2, padding="valid", activation="relu"))
    model.add(MaxPool1D(pool_size=3, strides=2))
    model.add(Conv1D(filters=384, kernel_size=3, strides=1, padding="same", activation="relu"))
    model.add(Conv1D(filters=384, kernel_size=3, strides=1, padding="same", activation="relu"))
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"))
    model.add(MaxPool1D(pool_size=3, strides=2))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(num_classes, activation="softmax"))
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------ZFNET---------------------------------')
    model.summary()
    return model

# ResNEt 18
def residual_block_18(x, filters, kernel_size=3, strides=1):
    # Shortcut
    shortcut = x
    # First convolution
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # Second convolution
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    # Adjusting the shortcut connection if needed
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    # Adding shortcut
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def ResNet18(input_shape, num_classes=6):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    # Stage 1
    x = residual_block_18(x, filters=64)
    x = residual_block_18(x, filters=64)
    # Stage 2
    x = residual_block_18(x, filters=128, strides=2)
    x = residual_block_18(x, filters=128)
    # Stage 3
    x = residual_block_18(x, filters=256, strides=2)
    x = residual_block_18(x, filters=256)
    # Stage 4
    x = residual_block_18(x, filters=512, strides=2)
    x = residual_block_18(x, filters=512)
    # Global average pooling
    x = GlobalAveragePooling1D()(x)
    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)
    # Model definition
    model = Model(inputs=inputs, outputs=outputs)
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------RESNET 18---------------------------------')
    model.summary()
    return model


# Resnet 34
def residual_block_34(x, filters, kernel_size=3, strides=1):
    # Shortcut
    shortcut = x
    # First convolution
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    # Second convolution
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    # Adjusting the shortcut connection if needed
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    # Adding shortcut
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def ResNet34(input_shape, num_classes=6):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    # First stage (with one residual block)
    x = residual_block_34(x, filters=64)
    # Second stage (with two residual blocks)
    x = residual_block_34(x, filters=64)
    x = residual_block_34(x, filters=64)
    # Third stage (with three residual blocks)
    x = residual_block_34(x, filters=128, strides=2)
    x = residual_block_34(x, filters=128)
    x = residual_block_34(x, filters=128)
    # Fourth stage (with five residual blocks)
    x = residual_block_34(x, filters=256, strides=2)
    x = residual_block_34(x, filters=256)
    x = residual_block_34(x, filters=256)
    x = residual_block_34(x, filters=256)
    x = residual_block_34(x, filters=256)
    # Fifth stage (with two residual blocks)
    x = residual_block_34(x, filters=512, strides=2)
    x = residual_block_34(x, filters=512)
    # Global average pooling
    x = GlobalAveragePooling1D()(x)
    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------RESNET 34---------------------------------')
    model.summary()
    return model

# AlexNet
def AlexNet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=96, kernel_size=11, strides=4, padding='same', activation='relu')(inputs)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    x = Conv1D(filters=256, kernel_size=5, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    x = Conv1D(filters=384, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=384, kernel_size=3, padding='same', activation='relu')(x)
    x = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
    x = MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print('---------------------------------ALEXNET---------------------------------')
    model.summary()
    return model

# RCCGNET
def conv_block(filters, x):
    x = Conv1D(filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def residual_block(filters, x):
    _, length, channels = x.shape
    r0 = AveragePooling1D(pool_size=length)(x)
    r0 = Conv1D(filters, 1, activation='sigmoid', padding='same')(r0)
    r1 = Multiply()([x, r0])
    return r1

def scr_block(x, y):
    channel = x.shape[-1]
    concat1 = Concatenate()([x, y])
    conv3 = conv_block(channel, concat1)
    res_x = residual_block(channel, conv3)
    return res_x

def resnet_block(block_input, filters):
    conv1 = conv_block(filters, block_input)
    conv2 = scr_block(block_input, conv1)
    sum_layer = Add()([conv2, block_input])
    res_out = Activation('relu')(sum_layer)
    return res_out

def rccgnet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    conv1 = conv_block(16, input_layer)
    scr1 = resnet_block(conv1, 16)
    scr2 = resnet_block(scr1, 16)
    pool1 = AveragePooling1D(2)(scr2)
    conv2 = conv_block(32, pool1)
    #scr3 = resnet_block(conv2, 32)
    #scr4 = resnet_block(scr3, 32)
    #pool2 = MaxPooling1D(2)(scr4)
    #conv3 = conv_block(64, pool2)
    scr5 = resnet_block(conv2, 64)
    scr6 = resnet_block(scr5, 64)
    pool3 = GlobalAveragePooling1D()(scr6)
    dense1 = Dense(128, activation='relu')(pool3)
    dense2 = Dense(128, activation='relu')(dense1)
    output = Dense(num_classes, activation='softmax')(dense2)
    model = Model(inputs=input_layer, outputs=output)
    print('---------------------------------RCCGNET---------------------------------')
    model.summary()
    return model

## Declarations

In [ ]:
# model_cnn = CNN_model0(input_shape, num_classes)
# model_vgg = VGG_model(input_shape, num_classes)
# model_zf = ZF_model(input_shape, num_classes)
# model_resnet_18 = ResNet18(input_shape, num_classes)
# model_resnet_34 = ResNet34(input_shape, num_classes)
# model_alexnet = AlexNet(input_shape, num_classes)
# model_rccgnet = rccgnet()

fold_no = 1
cv_scores_cnn = []  # To store accuracy of each fold
cv_f1_scores_cnn = []  # To store F1 score of each fold
tt_cnn = []

cv_scores_vgg = []  # To store accuracy of each fold
cv_f1_scores_vgg = []  # To store F1 score of each fold
tt_vgg = []

cv_scores_zf = []  # To store accuracy of each fold
cv_f1_scores_zf = []  # To store F1 score of each fold
tt_zf = []

cv_scores_resnet_18 = []  # To store accuracy of each fold
cv_f1_scores_resnet_18 = []  # To store F1 score of each fold
tt_resnet_18 = []

cv_scores_resnet_34 = []  # To store accuracy of each fold
cv_f1_scores_resnet_34 = []  # To store F1 score of each fold
tt_resnet_34 = []

cv_scores_alexnet = []  # To store accuracy of each fold
cv_f1_scores_alexnet = []  # To store F1 score of each fold
tt_alexnet = []

cv_scores_rccgnet = []  # To store accuracy of each fold
cv_f1_scores_rccgnet = []  # To store F1 score of each fold
tt_rccgnet = []

## 5-fold cross-validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# Loop over each fold
for train_index, val_index in kf.split(X_train):
    # Split data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_val[val_index]

    # Reshape the data as required by the model
    X_train_fold_reshaped = X_train_fold.reshape((-1,X_train_fold.shape[1], X_train_fold.shape[2]))
    X_val_fold_reshaped = X_val_fold.reshape((-1, X_val_fold.shape[1],X_val_fold.shape[2]))

    # Initialize a new model for each fold
    model_cnn = CNN_model0(input_data.shape[1:], num_classes)
    model_vgg = VGG_model(input_data.shape[1:], num_classes)
    model_zf = ZF_model(input_data.shape[1:], num_classes)
    model_resnet_18 = ResNet18(input_data.shape[1:], num_classes)
    model_resnet_34 = ResNet34(input_data.shape[1:], num_classes)
    model_alexnet = AlexNet(input_data.shape[1:], num_classes)
    model_rccgnet = rccgnet(input_data.shape[1:], num_classes)

    print(f'Training fold {fold_no}...')

    # Train the model
    st=datetime.now()
    history_cnn = model_cnn.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_cnn.append(datetime.now()-st)

    st=datetime.now()
    history_vgg = model_vgg.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_vgg.append(datetime.now()-st)

    st=datetime.now()
    history_zf = model_zf.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_zf.append(datetime.now()-st)

    st=datetime.now()
    history_resnet_18 = model_resnet_18.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_resnet_18.append(datetime.now()-st)

    st=datetime.now()
    history_resnet_34 = model_resnet_34.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_resnet_34.append(datetime.now()-st)

    st=datetime.now()
    history_alexnet = model_alexnet.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_alexnet.append(datetime.now()-st)

    st=datetime.now()
    history_rccgnet = model_rccgnet.fit(X_train_fold_reshaped, y_train_fold, epochs=65, validation_data=(X_val_fold_reshaped, y_val_fold))
    tt_rccgnet.append(datetime.now()-st)

    # Evaluate the model on the validation data for this fold
    y_val_true = np.argmax(y_val_fold, axis=1)

    # CNN
    scores_1 = model_cnn.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_cnn.metrics_names[1]} of {scores_1[1]}; loss of {scores_1[0]}')
    cv_scores_cnn.append(scores_1[1])
    y_val_pred_cnn = model_cnn.predict(X_val_fold_reshaped)
    y_val_pred_cnn = np.argmax(y_val_pred_cnn, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_cnn, average='weighted')
    cv_f1_scores_cnn.append(f1)

    # VGG
    scores_2 = model_vgg.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_vgg.metrics_names[1]} of {scores_2[1]}; loss of {scores_2[0]}')
    cv_scores_vgg.append(scores_2[1])
    y_val_pred_vgg = model_vgg.predict(X_val_fold_reshaped)
    y_val_pred_vgg = np.argmax(y_val_pred_vgg, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_vgg, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_vgg.append(f1)

    # ZF
    scores_3 = model_zf.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_zf.metrics_names[1]} of {scores_3[1]}; loss of {scores_3[0]}')
    cv_scores_zf.append(scores_3[1])
    y_val_pred_zf = model_zf.predict(X_val_fold_reshaped)
    y_val_pred_zf = np.argmax(y_val_pred_zf, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_zf, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_zf.append(f1)

    # resnet 18
    scores_4 = model_resnet_18.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_resnet_18.metrics_names[1]} of {scores_4[1]}; loss of {scores_4[0]}')
    cv_scores_resnet_18.append(scores_4[1])
    y_val_pred_resnet_18 = model_resnet_18.predict(X_val_fold_reshaped)
    y_val_pred_resnet_18 = np.argmax(y_val_pred_resnet_18, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_resnet_18, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_resnet_18.append(f1)

    # resnet 34
    scores_5 = model_resnet_34.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_resnet_34.metrics_names[1]} of {scores_5[1]}; loss of {scores_5[0]}')
    cv_scores_resnet_34.append(scores_5[1])
    y_val_pred_resnet_34 = model_resnet_34.predict(X_val_fold_reshaped)
    y_val_pred_resnet_34 = np.argmax(y_val_pred_resnet_34, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_resnet_34, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_resnet_34.append(f1)

    # Alexnet
    scores_6 = model_alexnet.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_alexnet.metrics_names[1]} of {scores_6[1]}; loss of {scores_6[0]}')
    cv_scores_alexnet.append(scores_6[1])
    y_val_pred_alexnet = model_alexnet.predict(X_val_fold_reshaped)
    y_val_pred_alexnet = np.argmax(y_val_pred_alexnet, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_alexnet, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_alexnet.append(f1)

    # RCCGNET
    scores_7 = model_rccgnet.evaluate(X_val_fold_reshaped, y_val_fold, verbose=1)
    print(f'Score for fold {fold_no}: {model_rccgnet.metrics_names[1]} of {scores_7[1]}; loss of {scores_7[0]}')
    cv_scores_rccgnet.append(scores_7[1])
    y_val_pred_rccgnet = model_rccgnet.predict(X_val_fold_reshaped)
    y_val_pred_rccgnet = np.argmax(y_val_pred_rccgnet, axis=1)
    f1 = f1_score(y_val_true, y_val_pred_rccgnet, average='weighted')
    print(f"F1 score for fold {fold_no}: {f1}")
    cv_f1_scores_rccgnet.append(f1)


    fold_no += 1


## Report

In [ ]:
print('::::::::::::::::::REPORT:::::::::::::::::::')

# cnn
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_cnn)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_cnn)}")
print(f"Average training time over 5 folds: {np.mean(tt_cnn)}")

# vgg
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_vgg)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_vgg)}")
print(f"Average training time over 5 folds: {np.mean(tt_vgg)}")

# ZF
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_zf)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_zf)}")
print(f"Average training time over 5 folds: {np.mean(tt_zf)}")

# resnet-18
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_resnet_18)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_resnet_18)}")
print(f"Average training time over 5 folds: {np.mean(tt_resnet_18)}")

# resnet-34
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_resnet_34)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_resnet_34)}")
print(f"Average training time over 5 folds: {np.mean(tt_resnet_34)}")

# alexnet
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_alexnet)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_alexnet)}")
print(f"Average training time over 5 folds: {np.mean(tt_alexnet)}")

# rcc
print(f"Average validation accuracy over 5 folds: {np.mean(cv_scores_rccgnet)}")
print(f"Average validation F1 score over 5 folds: {np.mean(cv_f1_scores_rccgnet)}")
print(f"Average training time over 5 folds: {np.mean(tt_rccgnet)}")